In [ ]:
import os
import time
import math
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
from itertools import combinations

# 1) Load the full population dataset
x_population = pd.read_csv("data/h_population.csv")

# 2) Encode each categorical column as integer labels
label_encoders = {}
for col in x_population.columns:
for col in x_population.columns:
    le = LabelEncoder()
    x_population[col] = le.fit_transform(x_population[col])
    label_encoders[col] = le

# 3) Convert columns to pandas 'category' dtype (expected by OneHotEncoder)
x_population = x_population.apply(lambda col: col.astype("category"))

# 4) Create and fit the OneHotEncoder
#    Note: use sparse_output=False in scikit-learn >=1.2
enc = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
enc.fit(x_population)

# 5) Load your actual sample data
# # Example of Spliting 5% Training data
# x_train_split = train_test_split(
#     x_population, test_size=0.05, shuffle=True, random_state=1004
# )[1]
x_train_split = pd.read_csv("data/h_sample.csv")

# 6) Apply the same LabelEncoder mappings to the sample data
for col in x_train_split.columns:
    x_train_split[col] = label_encoders[col].transform(x_train_split[col])

# 7) (Optional) Convert sample data to 'category' dtype as well
x_train_split = x_train_split.apply(lambda col: col.astype("category"))

# 8) Perform one-hot encoding on the sample
x_train_encoded = enc.transform(x_train_split)

# 9) Convert the encoded array into a PyTorch tensor
x_train_tensor = torch.tensor(x_train_encoded, dtype=torch.float32)

print("Training data shape:", x_train_tensor.shape)


Training data shape: torch.Size([53315, 70])


In [2]:

##########################
# 2. MODEL DEFINITIONS
##########################
class Generator(nn.Module):
    def __init__(self, latent_dim, hidden_dims, cat_dims):
        super().__init__()
        layers = []
        in_dim = latent_dim
        for dim in hidden_dims:
            layers.append(nn.Linear(in_dim, dim))
            layers.append(nn.ReLU())
            layers.append(nn.BatchNorm1d(dim))
            in_dim = dim
        self.hidden = nn.Sequential(*layers)

        # One output "head" per categorical column
        self.heads = nn.ModuleList([
            nn.Linear(hidden_dims[-1], cd) for cd in cat_dims
        ])

    def forward(self, noise):
        h = self.hidden(noise)
        outputs = []
        for head in self.heads:
            logits = head(h)
            probs = F.softmax(logits, dim=1)
            outputs.append(probs)
        return torch.cat(outputs, dim=1)

class Critic(nn.Module):
    def __init__(self, input_dim, hidden_dims):
        super().__init__()
        layers = []
        in_dim = input_dim
        for dim in hidden_dims:
            layers.append(nn.Linear(in_dim, dim))
            layers.append(nn.LeakyReLU(0.2))
            in_dim = dim
        layers.append(nn.Linear(hidden_dims[-1], 1))
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

def gradient_penalty(critic, real_data, fake_data, device):
    batch_size = real_data.size(0)
    epsilon = torch.rand(batch_size, 1, device=device)
    epsilon = epsilon.expand_as(real_data)

    interpolates = epsilon * real_data + (1 - epsilon) * fake_data
    interpolates.requires_grad_(True)

    critic_interpolates = critic(interpolates)
    grad_outputs = torch.ones_like(critic_interpolates)

    gradients = torch.autograd.grad(
        outputs=critic_interpolates,
        inputs=interpolates,
        grad_outputs=grad_outputs,
        create_graph=True,
        retain_graph=True
    )[0]

    gradients = gradients.view(batch_size, -1)
    gradient_norm = gradients.norm(2, dim=1)
    return ((gradient_norm - 1) ** 2).mean()



In [3]:
##########################
# 3. Utils
##########################

def wide_to_long(samples_pop, cat_boundaries, enc):
    """
    Converts the wide one-hot output to discrete categories
    by sampling from each column's predicted probabilities.
    """
    resamples = []
    for row_idx in range(samples_pop.shape[0]):
        row_probs = samples_pop[row_idx]
        resample_row = []
        for i in range(len(cat_boundaries) - 1):
            start = cat_boundaries[i]
            end   = cat_boundaries[i+1]
            probs = row_probs[start:end]
            sum_p = probs.sum()
            if sum_p > 0:
                probs /= sum_p
            else:
                probs = np.ones_like(probs) / len(probs)

            chosen_idx = np.random.choice(len(probs), p=probs)
            original_value = enc.categories_[i][chosen_idx]
            resample_row.append(original_value)
        resamples.append(resample_row)

    df = pd.DataFrame(resamples, columns=enc.feature_names_in_)
    return df.apply(lambda x: x.astype('category'))

def SRMSE(x_population: pd.DataFrame, resamples: pd.DataFrame):
    """Caculate Marginal·Bivariate SRMSE"""
    sam_marg_cnt, resam_marg_cnt = [], []
    for col in x_population.columns:
        pop_series = x_population[col].dropna()
        syn_series = resamples[col].dropna()

        tab = (pd
               .merge(syn_series.value_counts().rename('syn'),
                      pop_series.value_counts().rename('pop'),
                      left_index=True, right_index=True, how='outer')
               .fillna(0))
        sam_marg_cnt.append(tab['pop'].values / (len(pop_series) or 1))
        resam_marg_cnt.append(tab['syn'].values / (len(syn_series) or 1))

    sam_marg_cnt = np.concatenate(sam_marg_cnt) if sam_marg_cnt else np.array([])
    resam_marg_cnt = np.concatenate(resam_marg_cnt) if resam_marg_cnt else np.array([])

    if sam_marg_cnt.size:
        rmse_mar  = np.linalg.norm(sam_marg_cnt - resam_marg_cnt) / np.sqrt(len(sam_marg_cnt))
        ybar_mar  = sam_marg_cnt.mean()
        srmse_mar = rmse_mar / ybar_mar if ybar_mar else np.nan
    else:
        srmse_mar = np.nan

    # ── Bivariate ──────────────────────────────────────────
    sam_bi_cnt, resam_bi_cnt = [], []
    for c1, c2 in combinations(x_population.columns, 2):
        pop_pair = x_population[[c1, c2]].dropna()
        syn_pair = resamples[[c1, c2]].dropna()
        if pop_pair.empty or syn_pair.empty:
            continue  

        tab = (pd
               .merge(pd.crosstab(syn_pair[c1], syn_pair[c2]).stack().rename('syn'),
                      pd.crosstab(pop_pair[c1], pop_pair[c2]).stack().rename('pop'),
                      left_index=True, right_index=True, how='outer')
               .fillna(0))
        sam_bi_cnt.append(tab['pop'].values / (len(pop_pair) or 1))
        resam_bi_cnt.append(tab['syn'].values / (len(syn_pair) or 1))

    sam_bi_cnt  = np.concatenate(sam_bi_cnt)  if sam_bi_cnt  else np.array([])
    resam_bi_cnt = np.concatenate(resam_bi_cnt) if resam_bi_cnt else np.array([])

    if sam_bi_cnt.size:
        rmse_bi  = np.linalg.norm(sam_bi_cnt - resam_bi_cnt) / np.sqrt(len(sam_bi_cnt))
        ybar_bi  = sam_bi_cnt.mean()
        srmse_bi = rmse_bi / ybar_bi if ybar_bi else np.nan
    else:
        srmse_bi = np.nan

    return srmse_mar, srmse_bi


def calculate_precision_recall(population_df: pd.DataFrame,
                               generated_df: pd.DataFrame):
    """Precision·Recall·F1 and combinations"""
    pop_df = population_df.dropna()
    gen_df = generated_df.dropna()
    if pop_df.empty or gen_df.empty:
        return dict(precision=0, recall=0, f1_score=0,
                    unique_combinations={'population': 0, 'generated': 0},
                    matching_combinations={'unique_types': 0, 'total_count': 0})

    pop_tuples = pop_df.apply(tuple, axis=1)
    gen_tuples = gen_df.apply(tuple, axis=1)

    pop_set, gen_set = set(pop_tuples), set(gen_tuples)

    precision = (gen_tuples.isin(pop_set)).mean()
    recall    = (pop_tuples.isin(gen_set)).mean()
    f1        = 2*precision*recall/(precision+recall) if precision+recall else 0

    matching_unique = pop_set & gen_set
    gen_counts      = gen_tuples.value_counts()
    match_row_cnt   = sum(gen_counts[t] for t in matching_unique)

    return {
        'precision' : round(float(precision), 4),
        'recall'    : round(float(recall),    4),
        'f1_score'  : round(float(f1),        4),
        'unique_combinations': {
            'population': int(pop_tuples.nunique()),
            'generated' : int(gen_tuples.nunique())
        },
        'matching_combinations': {
            'unique_types': len(matching_unique),
            'total_count' : int(match_row_cnt)
        }
    }

def evaluate_coverage(
    x_population: pd.DataFrame,
    x_sample: pd.DataFrame,
    generator,
    cat_boundaries,
    enc,
    device,
    latent_dim: int,
    batch_size: int = 256,
):
    """
    1) Generate the same number of synthetic samples as the population
    2) Compute STZ/SAZ ratios, SRMSE, and Precision/Recall/F1
    3) Return a dictionary containing all metrics
    """
    # ── 1. Synthetic data generation ───────────────────────────────────────
    generator.eval().to(device)
    n_gen = len(x_population)
    results, done = [], 0
    t0 = time.time()

    with tqdm(total=n_gen, desc="Generating") as bar:
        while done < n_gen:
            cur = min(batch_size, n_gen - done)
            noise = torch.randn(cur, latent_dim, device=device)
            with torch.no_grad():
                fake = generator(noise).cpu().numpy()
            df_cat = wide_to_long(fake, cat_boundaries, enc)
            results.append(df_cat)
            done += cur
            bar.update(cur)

    resamples = pd.concat(results, ignore_index=True)
    infer_sec = time.time() - t0

    # ── 2. Common preprocessing (string conversion & column alignment) ─────
    pop_df = x_population.astype(str).reset_index(drop=True)
    sam_df = x_sample.astype(str).reset_index(drop=True)
    gen_df = resamples.astype(str).reindex(columns=pop_df.columns)

    # ── 3-A. STZ / SAZ computation ─────────────────────────────────────────
    pop_key = pop_df.agg("".join, axis=1)
    sam_key = sam_df.agg("".join, axis=1)
    gen_key = gen_df.agg("".join, axis=1)

    STZ = gen_df[~gen_key.isin(sam_key) & ~gen_key.isin(pop_key)]
    SAZ = gen_df[~gen_key.isin(sam_key) & gen_key.isin(pop_key)]
    stz_ratio = round(len(STZ) / len(gen_df), 4)
    saz_ratio = round(len(SAZ) / len(gen_df), 4)

    # ── 3-B. SRMSE computation ─────────────────────────────────────────────
    srmse_mar, srmse_bi = SRMSE(pop_df, gen_df)

    # ── 3-C. Precision, Recall, F1 computation ─────────────────────────────
    pr = calculate_precision_recall(pop_df, gen_df)

    # ── 4. Packaging results ───────────────────────────────────────────────
    metrics = {
        "stz_ratio": stz_ratio,
        "saz_ratio": saz_ratio,
        "srmse_marginal": srmse_mar,
        "srmse_bivariate": srmse_bi,
        "precision": pr["precision"],
        "recall": pr["recall"],
        "f1_score": pr["f1_score"],
        "unique_combinations": pr["unique_combinations"],
        "matching_combinations": pr["matching_combinations"],
        "infer_time_sec": round(infer_sec, 2),
    }
    return metrics

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"

cat_dims = [x_population[col].nunique() for col in x_population.columns]
total_output_dim = sum(cat_dims)

latent_dim = 256
hidden_dims = [256, 256, 256]
batch_size = 256
n_critic = 3
lambda_gp = 10
epochs = 250

generator = Generator(latent_dim, hidden_dims, cat_dims).to(device)
critic    = Critic(total_output_dim, hidden_dims).to(device)

optimizer_G = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.9))
optimizer_C = optim.Adam(critic.parameters(),   lr=0.0002, betas=(0.5, 0.9))

# Prepare a DataLoader to iterate over the entire training set each epoch
train_dataset = TensorDataset(x_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

os.makedirs("Vanilla-WGAN", exist_ok=True)
best_g_loss = float("inf")
save_interval = 50

train_start_time = time.time()

for epoch in tqdm(range(epochs), desc="Training Progress"):
    epoch_loss_G = 0.0
    epoch_loss_C = 0.0
    batch_count = 0

    for real_data_tuple in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}", leave=False):
        real_data = real_data_tuple[0].to(device)
        batch_count += 1

        # (A) Critic updates (n_critic times)
        for _ in range(n_critic):
            noise = torch.randn(real_data.size(0), latent_dim, device=device)
            with torch.no_grad():
                fake_data = generator(noise)

            optimizer_C.zero_grad()
            real_score = critic(real_data).mean()
            fake_score = critic(fake_data).mean()
            gp = gradient_penalty(critic, real_data, fake_data, device)
            d_loss = -real_score + fake_score + lambda_gp * gp
            d_loss.backward()
            optimizer_C.step()
            epoch_loss_C += d_loss.item()

        # (B) Generator update (1 step)
        noise = torch.randn(real_data.size(0), latent_dim, device=device)
        fake_data = generator(noise)
        optimizer_G.zero_grad()
        g_loss = -critic(fake_data).mean()
        g_loss.backward()
        optimizer_G.step()
        epoch_loss_G += g_loss.item()

    # Average losses across mini-batches
    avg_loss_G = epoch_loss_G / batch_count
    avg_loss_C = epoch_loss_C / (batch_count * n_critic)

    if (epoch + 1) % 10 == 0:
        tqdm.write(f"Epoch [{epoch+1}/{epochs}] | D Loss: {avg_loss_C:.4f} | G Loss: {avg_loss_G:.4f}")

    # Check if generator improved => save "best"
    if avg_loss_G < best_g_loss:
        best_g_loss = avg_loss_G
        torch.save(generator.state_dict(), "Vanilla-WGAN/generator_best.pth")
        torch.save(critic.state_dict(),    "Vanilla-WGAN/critic_best.pth")
        tqdm.write(f"  [Epoch {epoch+1}] Best model saved (G loss = {avg_loss_G:.4f})")

    # Periodic checkpoint
    if (epoch + 1) % save_interval == 0:
        torch.save(generator.state_dict(), f"Vanilla-WGAN/generator_epoch_{epoch+1}.pth")
        torch.save(critic.state_dict(),    f"Vanilla-WGAN/critic_epoch_{epoch+1}.pth")
        tqdm.write(f"  [Epoch {epoch+1}] Checkpoint saved.")

train_end_time = time.time()
train_duration = train_end_time - train_start_time
print("\nTraining complete.")
print(f"Total training time: {train_duration:.2f} seconds")


Training Progress:   0%|                                                                       | 0/250 [00:00<?, ?it/s]
C:\Users\euijin\anaconda3\envs\research_py311\Lib\site-packages\torch\autograd\graph.py:823: UserWarning: Attempting to run cuBLAS, but there was no current CUDA context! Attempting to set the primary context... (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\cuda\CublasHandlePool.cpp:180.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass

Training Progress:   0%|▎                                                              | 1/250 [00:07<29:51,  7.20s/it]

  [Epoch 1] Best model saved (G loss = -0.6804)



Training Progress:   1%|▌                                                              | 2/250 [00:14<30:41,  7.43s/it]

  [Epoch 2] Best model saved (G loss = -0.8964)



Training Progress:   1%|▊                                                              | 3/250 [00:22<30:44,  7.47s/it]

...

Epoch [240/250] | D Loss: -0.0845 | G Loss: -1.5939



Training Progress:  98%|███████████████████████████████████████████████████████████▊ | 245/250 [31:34<00:38,  7.74s/it]

  [Epoch 245] Best model saved (G loss = -1.6354)



Training Progress: 100%|█████████████████████████████████████████████████████████████| 250/250 [32:13<00:00,  7.74s/it]

Epoch [250/250] | D Loss: -0.0841 | G Loss: -1.6123
  [Epoch 250] Checkpoint saved.

Training complete.
Total training time: 1933.91 seconds


In [5]:
generator = Generator(latent_dim, hidden_dims, cat_dims).to(device)
generator.load_state_dict(torch.load("Vanilla-WGAN/generator_best.pth", map_location=device))
generator.eval()  # Set to evaluation mode

Generator(
  (hidden): Sequential(
    (0): Linear(in_features=256, out_features=256, bias=True)
    (1): ReLU()
    (2): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Linear(in_features=256, out_features=256, bias=True)
    (4): ReLU()
    (5): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): Linear(in_features=256, out_features=256, bias=True)
    (7): ReLU()
    (8): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (heads): ModuleList(
    (0): Linear(in_features=256, out_features=17, bias=True)
    (1): Linear(in_features=256, out_features=2, bias=True)
    (2): Linear(in_features=256, out_features=5, bias=True)
    (3): Linear(in_features=256, out_features=6, bias=True)
    (4-5): 2 x Linear(in_features=256, out_features=2, bias=True)
    (6): Linear(in_features=256, out_features=4, bias=True)
    (7): Linear(in_features=256, out_features=9, bias=True)
    (8)

In [6]:
# Compute categorical boundaries
cat_boundaries = [0] + list(np.cumsum(cat_dims))

evaluate_coverage(
    x_population=x_population,
    x_sample=x_train_split, 
    generator=generator,
    cat_boundaries=cat_boundaries,
    enc=enc,
    device=device,
    latent_dim=latent_dim,
    batch_size=256
)


Generating: 100%|██████████████████████████████████████████████████████████| 1066319/1066319 [07:00<00:00, 2538.29it/s]


{'stz_ratio': 0.1862,
 'saz_ratio': 0.2942,
 'srmse_marginal': np.float64(0.032534788830581494),
 'srmse_bivariate': np.float64(0.09533603745985905),
 'precision': 0.8138,
 'recall': 0.8089,
 'f1_score': 0.8113,
 'unique_combinations': {'population': 264005, 'generated': 264006},
 'matching_combinations': {'unique_types': 111932, 'total_count': 867786},
 'infer_time_sec': 426.89}

In [ ]:
# Set device (cuda or cpu)
device = "cuda" if torch.cuda.is_available() else "cpu"

# Define latent_dim, hidden_dims, cat_dims, etc. (should match those used during training)
latent_dim = 256
hidden_dims = [256, 256, 256]
cat_dims = [x_population[col].nunique() for col in x_population.columns]

cat_boundaries = [0] + list(np.cumsum(cat_dims))


# Re-create the generator instance and load the best saved model weights
generator = Generator(latent_dim, hidden_dims, cat_dims).to(device)
generator.load_state_dict(torch.load("Vanilla-WGAN/generator_best.pth", map_location=device))
generator.eval()  # Set to evaluation mode

# Number of synthetic samples to generate (for example, generate as many as in the full population)
num_samples = x_population.shape[0]  # or any desired number

# Create a list to store DataFrame batches
generated_batches = []
batch_size = 256

# Generate synthetic data in batches
with torch.no_grad():
    for i in range(0, num_samples, batch_size):
        current_batch = min(batch_size, num_samples - i)
        # Sample random noise from a standard normal distribution: 
        # mathematically, z ~ N(0, I), where z ∈ ℝ^(latent_dim)
        noise = torch.randn(current_batch, latent_dim, device=device)
        # Pass noise through generator to get softmax probabilities:
        # h = f(noise) and for each categorical head, output probabilities: 
        # \(\hat{y} = \text{softmax}(W h + b)\)
        fake_output = generator(noise).cpu().numpy()
        # Convert wide one-hot outputs to categorical values using the helper function
        df_batch = wide_to_long(fake_output, cat_boundaries, enc)
        generated_batches.append(df_batch)

# Concatenate all batches into a single DataFrame
generated_data = pd.concat(generated_batches, ignore_index=True)


Synthetic data generated and saved to 'generated_synthetic_data.csv'


In [8]:
decoded = generated_data.copy()

for col in decoded.columns:
    le = label_encoders[col]
    decoded[col] = le.inverse_transform( decoded[col].astype(int).values )

print(decoded.head())


       Age  Gender  Homeincome  Hometype  CarOwn  Driver  Workdays  Worktype  \
0  [45,50)       1           3         1       1       1         2         6   
1  [40,45)       1           2         1       1       1         1         8   
2  [35,40)       2           2         4       1       1         4         4   
3  [70,75)       2           1         1       1       2         4         2   
4  [55,60)       1           3         1       1       1         3         2   

   Student  NumHH  KidinHH                ComMode     ComTime  
0        4      3        2                    Car        Peak  
1        4      3        2  Public Transportation        Peak  
2        4      4        1             No commute  No commute  
3        4      3        1             No commute  No commute  
4        4      3        2                Walking     NonPeak  


In [ ]:
decoded.to_csv("generated_synthetic_data_WGAN.csv", index=False)